# Parameter Tuning

In [1]:
# Global variables
RANDOM_SEED = 153
NUM_TRIALS = 100

In [2]:
# Imports
import joblib
import time
import numpy as np
import pandas as pd
import gc; gc.enable()

# Hide warnings
import warnings
warnings.filterwarnings('ignore')

# Speedup some scikit-learn algorithms
from sklearnex import patch_sklearn
patch_sklearn()
import sklearn

# Scoring and Cross-Validation
from sklearn.base import clone
from sklearn.metrics import roc_auc_score
from sklearn.utils.extmath import softmax

# Imputation and Preprocessing
from sklearn.preprocessing import RobustScaler
from category_encoders import WOEEncoder
from sklearn.impute import KNNImputer

# Pipeline Constructors
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer

# Models
from sklearn.linear_model import LogisticRegression, HuberRegressor
from xgboost import XGBClassifier

# Optuna
import optuna
from optuna.visualization import plot_param_importances, plot_parallel_coordinate
from optuna.pruners import PercentilePruner

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
submission = pd.read_csv('../input/tabular-playground-series-aug-2022/sample_submission.csv')
train = pd.read_csv('../input/tabular-playground-series-aug-2022/train.csv', index_col = 'id') 
test = pd.read_csv('../input/tabular-playground-series-aug-2022/test.csv', index_col = 'id')

# Remove target column
target = train['failure'].copy()
gc.collect()

train.head()

,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,measurement_3,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
id,,,,,,,,,,,,,,,,,,,,,
0,A,80.10,material_7,material_8,9,5,7,8,4,18.040,...,10.672,15.859,17.594,15.193,15.029,NaN,13.034,14.684,764.100,0
1,A,84.89,material_7,material_8,9,5,14,3,3,18.213,...,12.448,17.947,17.915,11.755,14.732,15.425,14.395,15.631,682.057,0
2,A,82.43,material_7,material_8,9,5,12,1,5,18.057,...,12.715,15.607,NaN,13.798,16.711,18.631,14.094,17.946,663.376,0
3,A,101.07,material_7,material_8,9,5,13,2,6,17.295,...,12.471,16.346,18.377,10.020,15.250,15.562,16.154,17.172,826.282,0
4,A,188.06,material_7,material_8,9,5,9,2,8,19.346,...,10.337,17.082,19.932,12.428,16.182,12.760,13.153,16.412,579.885,0


# Preprocessing

In [4]:
def preprocessing(df_train, df_test):
    data = pd.concat([df_train, df_test])
    
    data['m3_missing'] = data['measurement_3'].isnull().astype(np.int8)
    data['m5_missing'] = data['measurement_5'].isnull().astype(np.int8)
    data['area'] = data['attribute_2'] * data['attribute_3']
    feature = [f for f in df_test.columns if f.startswith('measurement') or f =='loading']

    full_fill_dict ={}
    full_fill_dict['measurement_17'] = {
        'A': ['measurement_5','measurement_6','measurement_8'],
        'B': ['measurement_4','measurement_5','measurement_7'],
        'C': ['measurement_5','measurement_7','measurement_8','measurement_9'],
        'D': ['measurement_5','measurement_6','measurement_7','measurement_8'],
        'E': ['measurement_4','measurement_5','measurement_6','measurement_8'],
        'F': ['measurement_4','measurement_5','measurement_6','measurement_7'],
        'G': ['measurement_4','measurement_6','measurement_8','measurement_9'],
        'H': ['measurement_4','measurement_5','measurement_7','measurement_8','measurement_9'],
        'I': ['measurement_3','measurement_7','measurement_8']
    }

    # collect the name of the next 10 best measurement columns sorted by correlation (except 17 already done above):
    col = [col for col in df_test.columns if 'measurement' not in col]+ ['loading','m3_missing','m5_missing']
    a = []
    b =[]
    for x in range(3,17):
        corr = np.absolute(data.drop(col, axis=1).corr()[f'measurement_{x}']).sort_values(ascending=False)
        a.append(np.round(np.sum(corr[1:4]),3)) # we add the 3 first lines of the correlation values to get the "most correlated"
        b.append(f'measurement_{x}')
    c = pd.DataFrame()
    c['Selected columns'] = b
    c['correlation total'] = a
    c = c.sort_values(by = 'correlation total',ascending=False).reset_index(drop = True)
    #print(f'Columns selected by correlation sum of the 3 first rows : ')
    #display(c.head(10))

    for i in range(10):
        measurement_col = 'measurement_' + c.iloc[i,0][12:] # we select the next best correlated column 
        fill_dict = {}
        for x in data.product_code.unique() : 
            corr = np.absolute(data[data.product_code == x].drop(col, axis=1).corr()[measurement_col]).sort_values(ascending=False)
            measurement_col_dic = {}
            measurement_col_dic[measurement_col] = corr[1:5].index.tolist()
            fill_dict[x] = measurement_col_dic[measurement_col]
        full_fill_dict[measurement_col] =fill_dict

    feature = [f for f in data.columns if f.startswith('measurement') or f=='loading']
    nullValue_cols = [col for col in df_train.columns if df_train[col].isnull().sum()!=0]

    for code in data.product_code.unique():
        for measurement_col in list(full_fill_dict.keys()):
            tmp = data[data.product_code == code]
            column = full_fill_dict[measurement_col][code]
            tmp_train = tmp[column+[measurement_col]].dropna(how='any')
            tmp_test = tmp[(tmp[column].isnull().sum(axis=1)==0)&(tmp[measurement_col].isnull())]

            model = HuberRegressor(epsilon=1.9)
            model.fit(tmp_train[column], tmp_train[measurement_col])
            data.loc[(data.product_code==code)&(data[column].isnull().sum(axis=1)==0)&(data[measurement_col].isnull()),measurement_col] = model.predict(tmp_test[column])

        # others NA columns:
        NA = data.loc[data["product_code"] == code,nullValue_cols ].isnull().sum().sum()
        model1 = KNNImputer(n_neighbors=3)
        data.loc[data.product_code==code, feature] = model1.fit_transform(data.loc[data.product_code==code, feature])

    data['measurement_avg'] = data[[f'measurement_{i}' for i in range(3, 17)]].mean(axis=1)
    df_train = data.iloc[:df_train.shape[0],:]
    df_test = data.iloc[df_train.shape[0]:,:]
    features = ['loading', 'attribute_0', 'measurement_17', 'measurement_0', 'measurement_1', 'measurement_2', 'area', 'm3_missing', 'm5_missing', 'measurement_avg']
    
    return df_train, df_test, features

In [5]:
%%time
train, test, features = preprocessing(train, test)

CPU times: user 40.5 s, sys: 30.2 s, total: 1min 10s
Wall time: 22.2 s


In [6]:
# https://www.kaggle.com/code/purist1024/principled-3-vs-2-cv-splitting-on-product-code
class ProductSplitter3v2():
    def split(self, X: pd.DataFrame, _y=None, _groups=None):
        indices = list(X.groupby("product_code").indices.values())
        for i in range(len(indices)):
            for j in range(i + 1, len(indices)):
                yield [
                    np.concatenate([ix for k, ix in enumerate(indices) if k not in [i, j]]),
                    np.concatenate([ix for k, ix in enumerate(indices) if k in [i, j]])
                ]

In [7]:
# CV scheme
cv = ProductSplitter3v2()
SPLITS = list(cv.split(train))

# Scoring Function

In [8]:
preprocessing = make_pipeline(
    make_column_transformer(
        (WOEEncoder(), ['attribute_0']),
        (FunctionTransformer(np.log1p), ['loading']),
        remainder = 'passthrough'
    ),
    RobustScaler()
)

In [9]:
# Percentile Pruner settings
pruner = PercentilePruner(
    percentile = 75,
    n_startup_trials = 15,
    n_warmup_steps = 5,
    interval_steps = 1,
    n_min_trials = 15,
)

In [10]:
# Scoring Function
def score(sklearn_model):

    # Stores predictions and scores
    test_preds = np.zeros((test.shape[0],)) 
    oof_preds = np.zeros((train.shape[0],))
    scores = np.zeros(len(SPLITS))
    total_time = 0
    
    # Stratified k-fold cross-validation
    start = time.time()
    for fold, (train_idx, valid_idx) in enumerate(SPLITS):

        # Training and Validation Sets
        X_train = train[features].iloc[train_idx].copy()
        X_valid = train[features].iloc[valid_idx].copy()
        y_train = target.iloc[train_idx].copy()
        y_valid = target.iloc[valid_idx].copy()

       # Define model and train
        model = make_pipeline(
            clone(preprocessing),
            clone(sklearn_model)
        )
        model.fit(X_train, y_train)

        # Get predictions
        valid_preds = model.predict_proba(X_valid)[:,1]
        test_preds += model.predict_proba(test[features])[:, 1] / len(SPLITS)
        oof_preds[valid_idx] = valid_preds
        scores[fold] = roc_auc_score(y_valid, valid_preds)
        end = time.time()
        total_time += (end-start) 
    print(f'Worst: {round(np.min(scores), 6)}')
    print(f'Average:{round(np.mean(scores), 6)}')
    print(f'Completed in {round((total_time) / 60, 2)} minutes')
    time.sleep(0.5)
    return np.mean(scores), test_preds

## XGBoost Linear

In [11]:
default_params = dict(            
    booster = 'gblinear',
    eval_metric = 'logloss',
    random_state = RANDOM_SEED,
)

In [12]:
def parameter_search(trials):
    
    # Optuna objective function
    def objective(trial):
        model_params = dict( 
            # default 0
            reg_lambda = trial.suggest_loguniform(
                "reg_lambda", 1e-10, 10
            ),
            # default 0
            reg_alpha = trial.suggest_loguniform(
                "reg_alpha", 1e-10, 10
            ),
            # default cyclic
            feature_selector = trial.suggest_categorical(
                "feature_selector",['cyclic','shuffle']
            ),
        )
        model = XGBClassifier(
            **default_params,
            **model_params
        )
        return score(model)[0]
    
    
    optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study = optuna.create_study(pruner = pruner,direction = "maximize")
    # Close to defaults
    study.enqueue_trial({
        'reg_lambda': 1e-10, 
        'reg_alpha': 1e-10, 
        'feature_selector': 'cyclic'
    })
    study.optimize(objective, n_trials=trials)
    return study

In [13]:
# Hide output
study = parameter_search(NUM_TRIALS)

[I 2023-01-04 00:33:48,208] A new study created in memory with name: no-name-0e1b2a94-fa69-4f0f-b356-53a2bd8dd25f
[D 2023-01-04 00:33:48,210] Trial 0 popped from the trial queue.


Worst: 0.583475
Average:0.588787
Completed in 0.54 minutes


[I 2023-01-04 00:33:54,547] Trial 0 finished with value: 0.5887868422461884 and parameters: {'reg_lambda': 1e-10, 'reg_alpha': 1e-10, 'feature_selector': 'cyclic'}. Best is trial 0 with value: 0.5887868422461884.


Worst: 0.5
Average:0.5
Completed in 0.5 minutes


[I 2023-01-04 00:34:00,512] Trial 1 finished with value: 0.5 and parameters: {'reg_lambda': 4.068925596065909e-08, 'reg_alpha': 4.67743960192527, 'feature_selector': 'shuffle'}. Best is trial 0 with value: 0.5887868422461884.


...

[I 2023-01-04 00:44:16,241] Trial 97 finished with value: 0.589264531228828 and parameters: {'reg_lambda': 0.0023813680031867877, 'reg_alpha': 4.544219229199855e-09, 'feature_selector': 'shuffle'}. Best is trial 94 with value: 0.5892648893933595.


Worst: 0.58425
Average:0.589219
Completed in 0.55 minutes


[I 2023-01-04 00:44:22,501] Trial 98 finished with value: 0.589218810126881 and parameters: {'reg_lambda': 0.0048486186710239056, 'reg_alpha': 5.430049023887678e-09, 'feature_selector': 'shuffle'}. Best is trial 94 with value: 0.5892648893933595.


Worst: 0.584028
Average:0.589026
Completed in 0.52 minutes


[I 2023-01-04 00:44:28,679] Trial 99 finished with value: 0.5890261143453515 and parameters: {'reg_lambda': 0.017921696782501603, 'reg_alpha': 2.0126793149642251e-10, 'feature_selector': 'cyclic'}. Best is trial 94 with value: 0.5892648893933595.


In [14]:
print("Best Parameters:", study.best_params)

Best Parameters: {'reg_lambda': 0.002307467194699118, 'reg_alpha': 6.115133720223197e-09, 'feature_selector': 'shuffle'}


In [15]:
plot_parallel_coordinate(study)

In [16]:
submission['failure'] = score(
    XGBClassifier(**default_params, **study.best_params)
)[1]
submission.to_csv('xgboost_submission.csv', index=False)

Worst: 0.584349
Average:0.589265
Completed in 0.53 minutes


## Logistic Regression w/ L1 Regularlization

In [17]:
default_params = dict(            
    solver = 'liblinear',
    penalty = 'l1',
    max_iter = 500,
    random_state = RANDOM_SEED,
)

In [18]:
def parameter_search(trials):
    
    # Optuna objective function
    def objective(trial):
        model_params = dict( 
            # default 1.0
            C = trial.suggest_loguniform(
                "C", 1e-10, 100
            ),
        )
        model = LogisticRegression(
            **default_params,
            **model_params
        )
        return score(model)[0]
    
    
    optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study = optuna.create_study(pruner = pruner,direction = "maximize")
    # Close to defaults
    study.enqueue_trial({'C': 1.0})
    study.optimize(objective, n_trials=trials)
    return study

In [19]:
# Hide output
study = parameter_search(NUM_TRIALS)

[I 2023-01-04 00:44:35,414] A new study created in memory with name: no-name-271a13fb-2c88-4c03-893b-1c1c7efac892
[D 2023-01-04 00:44:35,417] Trial 0 popped from the trial queue.


Worst: 0.583658
Average:0.588886
Completed in 0.13 minutes


[I 2023-01-04 00:44:37,559] Trial 0 finished with value: 0.5888858513687089 and parameters: {'C': 1.0}. Best is trial 0 with value: 0.5888858513687089.


Worst: 0.584233
Average:0.589167
Completed in 0.11 minutes


[I 2023-01-04 00:44:39,297] Trial 1 finished with value: 0.5891671478696981 and parameters: {'C': 0.1956127925249981}. Best is trial 1 with value: 0.5891671478696981.


...

[I 2023-01-04 00:47:30,638] Trial 97 finished with value: 0.5891186098128389 and parameters: {'C': 0.024824965521133117}. Best is trial 95 with value: 0.5891843537250654.


Worst: 0.583907
Average:0.58864
Completed in 0.09 minutes


[I 2023-01-04 00:47:32,143] Trial 98 finished with value: 0.588640036136461 and parameters: {'C': 0.011316101283689525}. Best is trial 95 with value: 0.5891843537250654.


Worst: 0.584282
Average:0.589182
Completed in 0.11 minutes


[I 2023-01-04 00:47:33,840] Trial 99 finished with value: 0.5891818170867247 and parameters: {'C': 0.13352854881404533}. Best is trial 95 with value: 0.5891843537250654.


In [20]:
print("Best Parameters:", study.best_params)

Best Parameters: {'C': 0.1443247855892875}


In [21]:
plot_parallel_coordinate(study)

In [22]:
submission['failure'] = score(
    LogisticRegression(**default_params, **study.best_params)
)[1]
submission.to_csv('l1_submission.csv', index=False)

Worst: 0.584263
Average:0.589184
Completed in 0.11 minutes


# Logistic Regression w/ L2 Regularization

In [23]:
default_params = dict(            
    solver = 'liblinear',
    penalty = 'l2',
    max_iter = 500,
    random_state = RANDOM_SEED,
)

In [24]:
def parameter_search(trials):
    
    # Optuna objective function
    def objective(trial):
        model_params = dict( 
            # default 1.0
            C = trial.suggest_loguniform(
                "C", 1e-10, 100
            ),
        )
        model = LogisticRegression(
            **default_params,
            **model_params
        )
        return score(model)[0]
    
    
    optuna.logging.set_verbosity(optuna.logging.DEBUG)
    study = optuna.create_study(pruner = pruner,direction = "maximize")
    # Close to defaults
    study.enqueue_trial({'C': 1.0})
    study.optimize(objective, n_trials=trials)
    return study

In [25]:
# Hide output
study = parameter_search(NUM_TRIALS)

[I 2023-01-04 00:47:35,696] A new study created in memory with name: no-name-ade337a5-59f0-4944-a068-341167116888
[D 2023-01-04 00:47:35,698] Trial 0 popped from the trial queue.


Worst: 0.583366
Average:0.588677
Completed in 0.12 minutes


[I 2023-01-04 00:47:37,461] Trial 0 finished with value: 0.5886766795377172 and parameters: {'C': 1.0}. Best is trial 0 with value: 0.5886766795377172.


Worst: 0.583282
Average:0.58857
Completed in 0.11 minutes


[I 2023-01-04 00:47:39,186] Trial 1 finished with value: 0.5885703810531964 and parameters: {'C': 59.5071385590035}. Best is trial 0 with value: 0.5886766795377172.


...

[I 2023-01-04 00:50:16,794] Trial 97 finished with value: 0.5891361954427323 and parameters: {'C': 0.1004225201344895}. Best is trial 46 with value: 0.5895593435593651.


Worst: 0.584153
Average:0.589557
Completed in 0.11 minutes


[I 2023-01-04 00:50:18,454] Trial 98 finished with value: 0.5895573164275827 and parameters: {'C': 0.010139158799809865}. Best is trial 46 with value: 0.5895593435593651.


Worst: 0.584196
Average:0.5895
Completed in 0.11 minutes


[I 2023-01-04 00:50:20,153] Trial 99 finished with value: 0.5895001786373453 and parameters: {'C': 0.019989990552984466}. Best is trial 46 with value: 0.5895593435593651.


In [26]:
print("Best Parameters:", study.best_params)

Best Parameters: {'C': 0.009794986261873296}


In [27]:
plot_parallel_coordinate(study)

In [28]:
submission['failure'] = score(
    LogisticRegression(**default_params, **study.best_params)
)[1]
submission.to_csv('l2_submission.csv', index=False)

Worst: 0.584146
Average:0.589559
Completed in 0.11 minutes
